In [1]:
# one hot can not describes similarity

In [2]:
#跳字模型（Skip-gram）
#连续词袋模型(CBOW)
#概率为所有词成绩，转化成最小损失log函数计算求导求梯度
#降低复杂度使用近似训练方法，negative sampling 和hierachical softmax

In [3]:
import collections
import math
import random
import sys
import time
import os
import numpy as np
import torch
from torch import nn
import torch.utils.data as Data

sys.path.append("D:\CS\MachineLearning\Dive-into-DL-PyTorch-master\code") 
import d2lzh_pytorch as d2l
print(torch.__version__)

1.4.0+cpu


In [17]:
#preprocessing dataset
#use PTB (penn tree bank) dataset
assert 'ptb.train.txt' in os.listdir(r"D:\CS\MachineLearning\Dive-into-DL-PyTorch-master\data\ptb")
i=1
with open(r"D:\CS\MachineLearning\Dive-into-DL-PyTorch-master\data\ptb\ptb.train.txt",'r') as f:
    lines=f.readlines()
    raw_dataset=[st.split() for st in lines]
'# sentences: %d' % len(raw_dataset)

'# sentences: 42068'

In [21]:
for st in raw_dataset[:3]:
    print('# tokens:', len(st),st[:5])

# tokens: 24 ['aer', 'banknote', 'berlitz', 'calloway', 'centrust']
# tokens: 15 ['pierre', '<unk>', 'N', 'years', 'old']
# tokens: 11 ['mr.', '<unk>', 'is', 'chairman', 'of']


In [27]:
#count the numbers of each words
counter=collections.Counter([tk for st in raw_dataset for tk in st])
#only preserve the number>=5
counter=dict(filter(lambda x:x[1]>=5,counter.items()))
counter

{'pierre': 6,
 '<unk>': 45020,
 'N': 32481,
 'years': 1241,
 'old': 268,
 'will': 3270,
 'join': 45,
 'the': 50770,
 'board': 612,
 'as': 4833,
 'a': 21196,
 'nonexecutive': 6,
 'director': 359,
 'nov.': 259,
 'mr.': 4326,
 'is': 7337,
 'chairman': 635,
 'of': 24400,
 'n.v.': 13,
 'dutch': 28,
 'publishing': 64,
 'group': 928,
 'rudolph': 8,
 'and': 17474,
 'former': 306,
 'consolidated': 37,
 'gold': 165,
 'fields': 44,
 'plc': 114,
 'was': 4073,
 'named': 210,
 'this': 2438,
 'british': 337,
 'industrial': 243,
 'conglomerate': 22,
 'form': 115,
 'asbestos': 27,
 'once': 219,
 'used': 372,
 'to': 23638,
 'make': 646,
 'kent': 11,
 'cigarette': 18,
 'filters': 11,
 'has': 3494,
 'caused': 110,
 'high': 400,
 'percentage': 142,
 'cancer': 107,
 'deaths': 32,
 'among': 444,
 'workers': 247,
 'exposed': 19,
 'it': 6112,
 'more': 2065,
 'than': 1731,
 'ago': 468,
 'researchers': 85,
 'reported': 430,
 'fiber': 8,
 'unusually': 20,
 'enters': 9,
 'with': 4585,
 'even': 773,
 'brief': 34,
 

In [38]:
idx_to_token=[tk for tk,_ in counter.items()]#get the words list
token_to_idx={tk:idx for idx,tk in enumerate(idx_to_token)}#获得整数索引
dataset=[[token_to_idx[tk] for tk in st if tk in token_to_idx] for st in raw_dataset]#每一个句子中词的索引list集合
num_tokens=sum([len(st) for st in dataset])#总词数
'# tokens: %d' % num_tokens

'# tokens: 887100'

In [41]:
#sub sampling
def discard(idx):
    return random.uniform(0,1)<1 -math.sqrt(
    1e-4/counter[idx_to_token[idx]]*num_tokens)#二次采样公式，词出现频率除以总词数的倒数，词频率越大越容易丢弃，可以丢弃（the, a）这些没意义的词
subsampled_dataset=[[tk for tk in st if not discard(tk)] for st in dataset]
'tokens:%d' % sum([len(st) for st in subsampled_dataset])

'tokens:375674'

In [45]:
#compare the number before and after the subsampling
def compare_counts(token):
    return '# %s: before=%d, after=%d' % (token, sum(
        [st.count(token_to_idx[token]) for st in dataset]), sum(
        [st.count(token_to_idx[token]) for st in subsampled_dataset]))
compare_counts('the')                                        

'# the: before=50770, after=2158'

In [46]:
dataset

[[],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 2],
 [14, 1, 15, 16, 17, 1, 18, 7, 19, 20, 21],
 [22,
  1,
  2,
  3,
  4,
  23,
  24,
  16,
  17,
  25,
  26,
  27,
  28,
  29,
  30,
  10,
  11,
  12,
  17,
  31,
  32,
  33,
  34],
 [10,
  35,
  17,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  10,
  46,
  47,
  17,
  48,
  49,
  50,
  10,
  21,
  17,
  51,
  52,
  39,
  53,
  54,
  55,
  2,
  3,
  56,
  57,
  58],
 [7,
  36,
  59,
  1,
  15,
  60,
  1,
  37,
  53,
  61,
  7,
  1,
  62,
  63,
  64,
  65,
  39,
  53,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  57,
  73],
 [1,
  74,
  7,
  75,
  17,
  76,
  77,
  1,
  78,
  68,
  79,
  41,
  80,
  81,
  82,
  1,
  83,
  84,
  1,
  42,
  43,
  83,
  2],
 [85,
  86,
  87,
  88,
  58,
  54,
  55,
  10,
  89,
  56,
  7,
  90,
  91,
  92,
  83,
  93,
  94,
  76,
  95,
  96,
  17,
  97,
  10,
  98,
  99,
  39,
  100,
  76,
  101,
  39,
  7,
  102],
 [10, 1, 1, 73, 31, 15, 103, 4, 104],
 [105, 106, 107, 108, 3, 56, 109, 110,

In [51]:
#获得每一个中心词和相对应的背景词的索引
def get_centers_and_contexts(dataset,max_window_size):
    centers,contexts=[],[]
    for st in dataset:
        if len(st)<2:#每个句子至少有2个词才可以组成一对“中心词和背景词”
            continue
        centers+=st
        for center_i in range(len(st)):
            window_size=random.randint(1,max_window_size)
            indices=list(range(max(0,center_i-window_size),min(len(st),center_i+1+window_size)))#得到中心词窗口大小的索引
            
            indices.remove(center_i)#去掉中心词索引
            contexts.append([st[idx] for idx in indices])#得到背景词索引
    return centers,contexts

In [64]:
all_centers,all_contexts=get_centers_and_contexts(subsampled_dataset,5)

In [55]:
#manual test
tiny_dataset=[list(range(7)),list(range(7,10))]
print("dataset",tiny_dataset)
for center,context in zip(*get_centers_and_contexts(tiny_dataset,2)):
    print('center',center,"has contexts",context)

dataset [[0, 1, 2, 3, 4, 5, 6], [7, 8, 9]]
center 0 has contexts [1, 2]
center 1 has contexts [0, 2, 3]
center 2 has contexts [1, 3]
center 3 has contexts [2, 4]
center 4 has contexts [2, 3, 5, 6]
center 5 has contexts [3, 4, 6]
center 6 has contexts [5]
center 7 has contexts [8]
center 8 has contexts [7, 9]
center 9 has contexts [7, 8]


In [66]:
#negative sampling
def get_negatives(all_contexts,sampling_weights,K):
    all_negatives,neg_candidates,i=[],[],0
    population =list(range(len(sampling_weights)))
    for contexts in all_contexts:
        negatives=[]
        while len(negatives)<len(contexts)*K:
            if i==len(neg_candidates):
                #根据每个词的权重（sampling_weights)随机生成k个词的索引作为噪声词
                #为了高效计算，可以将k设得大一点
                i,neg_candidates=0,random.choices(
                population,sampling_weights,k=int(1e5))
            neg,i=neg_candidates[i],i+1
            #噪声词不能是背景词
            if neg not in set(contexts):
                negatives.append(neg)
        all_negatives.append(negatives)
    return all_negatives

In [67]:
sampling_weights=[counter[w]**0.75 for w in idx_to_token]
all_negatives=get_negatives(all_contexts,sampling_weights,5)#get corresponding negative sampling
all_negatives

[[1366,
  1,
  10,
  2850,
  39,
  619,
  2557,
  578,
  129,
  314,
  520,
  443,
  134,
  5484,
  5478,
  89,
  202,
  2015,
  4549,
  180],
 [1,
  170,
  6309,
  9240,
  10,
  156,
  335,
  222,
  6629,
  9293,
  3135,
  2587,
  386,
  9853,
  5972],
 [3227,
  1509,
  6798,
  1736,
  1268,
  825,
  2203,
  3084,
  4835,
  39,
  183,
  1,
  255,
  38,
  3258,
  6833,
  2070,
  422,
  2412,
  200],
 [506,
  23,
  83,
  4082,
  213,
  5393,
  6480,
  489,
  2753,
  1325,
  538,
  726,
  486,
  4037,
  2292,
  1004,
  7,
  1643,
  1931,
  779],
 [6751, 846, 105, 134, 2039],
 [1249,
  2369,
  7389,
  3758,
  269,
  413,
  1352,
  473,
  23,
  9493,
  1,
  371,
  7042,
  2488,
  7381],
 [2745,
  156,
  8298,
  2,
  9677,
  656,
  84,
  483,
  348,
  5084,
  8736,
  1760,
  2315,
  2132,
  287],
 [7854,
  7130,
  156,
  5021,
  1169,
  4205,
  3238,
  1717,
  39,
  4187,
  366,
  103,
  93,
  3184,
  10],
 [3295,
  1003,
  7,
  1987,
  8230,
  5013,
  144,
  15,
  1363,
  4590,
  105,
  52

In [75]:
#read dataset
class MyDataset(torch.utils.data.Dataset):
    def __init__(self,centers,contexts,negatives):
        assert len(centers)==len(contexts)==len(negatives)
        self.centers=centers
        self.contexts=contexts
        self.negatives=negatives
    def __getitem__(self,index):
        return (self.centers[index],self.contexts[index],self.negatives[index])
    def __len__(self):
        return len(self.centers)

In [83]:
# define batchify
def batchify(data):
    """用作DataLoader的参数collate_fn: 输入是个长为batchsize的list, 
    list中的每个元素都是Dataset类调用__getitem__得到的结果
    """
    max_len=max(len(c)+len(n) for _,c,n in data)
    centers,contexts_nagatives,masks,labels=[],[],[],[]
    for center,context,negative in data:
        cur_len=len(context)+len(negative)
        centers+=[center]
        contexts_nagatives+=[context+negative+[0]*(max_len-cur_len)]
        masks+=[[1]*cur_len+[0]*(max_len-cur_len)]
        labels+=[[1]*len(context)+[0]*(max_len-len(context))]
    return (torch.tensor(centers).view(-1,1),torch.tensor(contexts_nagatives),torch.tensor(masks),torch.tensor(labels))

In [84]:

batch_size=512
num_workers=0 if sys.platform.startswith("win32") else 4

dataset=MyDataset(all_centers,all_contexts,all_negatives)
data_iter=Data.DataLoader(dataset,batch_size,shuffle=True,collate_fn=batchify,num_workers=num_workers)
for batch in data_iter:
    for name,data in zip(["centers","contexts_negatives","masks","labels"],batch):
        print(name,"shape:",data.shape)
    break

centers shape: torch.Size([512, 1])
contexts_negatives shape: torch.Size([512, 60])
masks shape: torch.Size([512, 60])
labels shape: torch.Size([512, 60])


In [95]:
#skip-gram
#embedding
embed=nn.Embedding(num_embeddings=20,embedding_dim=4)
embed.weight

def skip_gram(center,contexts_and_negatives,embed_v,embed_u):
    v=embed_v(center)
    u=embed_u(contexts_and_negatives)
    pred=torch.bmm(v,u.permute(0,2,1))
    return pred

In [96]:
#train_model
class SigmoidBinaryCrossEntropyLoss(nn.Module):
    def __init__(self): # none mean sum
        super(SigmoidBinaryCrossEntropyLoss, self).__init__()
    def forward(self, inputs, targets, mask=None):
        """
        input – Tensor shape: (batch_size, len)
        target – Tensor of the same shape as input
        """
        inputs, targets, mask = inputs.float(), targets.float(), mask.float()
        res = nn.functional.binary_cross_entropy_with_logits(inputs, targets, reduction="none", weight=mask)
        return res.mean(dim=1)

loss = SigmoidBinaryCrossEntropyLoss()

In [97]:
pred = torch.tensor([[1.5, 0.3, -1, 2], [1.1, -0.6, 2.2, 0.4]])
# 标签变量label中的1和0分别代表背景词和噪声词
label = torch.tensor([[1, 0, 0, 0], [1, 1, 0, 0]])
mask = torch.tensor([[1, 1, 1, 1], [1, 1, 1, 0]])  # 掩码变量
loss(pred, label, mask) * mask.shape[1] / mask.float().sum(dim=1)

tensor([0.8740, 1.2100])

In [98]:
#nn model
embed_size = 100
net = nn.Sequential(
    nn.Embedding(num_embeddings=len(idx_to_token), embedding_dim=embed_size),
    nn.Embedding(num_embeddings=len(idx_to_token), embedding_dim=embed_size)
)

In [99]:
def train(net, lr, num_epochs):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print("train on", device)
    net = net.to(device)
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    for epoch in range(num_epochs):
        start, l_sum, n = time.time(), 0.0, 0
        for batch in data_iter:
            center, context_negative, mask, label = [d.to(device) for d in batch]

            pred = skip_gram(center, context_negative, net[0], net[1])

            # 使用掩码变量mask来避免填充项对损失函数计算的影响
            l = (loss(pred.view(label.shape), label, mask) *
                 mask.shape[1] / mask.float().sum(dim=1)).mean() # 一个batch的平均loss
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            l_sum += l.cpu().item()
            n += 1
        print('epoch %d, loss %.2f, time %.2fs'
              % (epoch + 1, l_sum / n, time.time() - start))

In [100]:
train(net, 0.01, 10)

train on cpu
epoch 1, loss 1.98, time 123.70s
epoch 2, loss 0.62, time 122.16s
epoch 3, loss 0.45, time 119.55s
epoch 4, loss 0.39, time 119.87s
epoch 5, loss 0.37, time 121.03s
epoch 6, loss 0.35, time 128.14s
epoch 7, loss 0.34, time 132.05s
epoch 8, loss 0.33, time 129.03s
epoch 9, loss 0.32, time 129.18s
epoch 10, loss 0.32, time 128.53s


In [104]:
#get similar tokens
def get_similar_tokens(query_token, k, embed):
    W = embed.weight.data
    x = W[token_to_idx[query_token]]
    # 添加的1e-9是为了数值稳定性
    cos = torch.matmul(W, x) / (torch.sum(W * W, dim=1) * torch.sum(x * x) + 1e-9).sqrt()
    _, topk = torch.topk(cos, k=k+1)
    topk = topk.cpu().numpy()
    for i in topk[1:]:  # 除去输入词
        print('cosine sim=%.3f: %s' % (cos[i], (idx_to_token[i])))

get_similar_tokens('chip',5, net[0])

cosine sim=0.482: intel
cosine sim=0.401: storage
cosine sim=0.401: second
cosine sim=0.399: palo
cosine sim=0.395: inventories
